In [2]:
import sys
sys.path.append("D:/File/大/大三下/poke")
# 导入 Flask 框架、request 对象和 secure_filename 方法
from flask import Flask, request, jsonify
from werkzeug.utils import secure_filename
import os
from Processing import Processing
from flask_cors import *

In [3]:
# 允许上传的文件类型，这里限制为 csv、xlsx 和 xls 格式的文件
ALLOWED_EXTENSIONS = set(['csv', 'xlsx', 'xls'])

# 检查上传的文件类型是否在 ALLOWED_EXTENSIONS 中
def isAllowedFile(filename):
    return '.'in filename and filename.split('.')[-1] in ALLOWED_EXTENSIONS

In [4]:
app=Flask(__name__)
# 设置文件上传保存路径，这里设置为当前文件夹下的 upload 文件夹
app.config['UPLOAD_FOLDER'] = 'upload/'
CORS(app, supports_credentials=True)

In [5]:
@app.route('/load_network',methods=['GET'])
def load_network():
    with open("network_data.json",'r', encoding='UTF-8') as f:
        load_dict = f.read()
    return load_dict

In [6]:
@app.route('/load_wordcloud',methods=['GET'])
def load_wordcloud():
    with open("wordcloud.json",'r', encoding='UTF-8') as f:
        load_dict = f.read()
    return load_dict

In [7]:
@app.route('/load_line',methods=['POST'])
def load_line():
    lst=request.get_json()

    import json
    with open('line_data.json','r', encoding='UTF-8')as f:
        data=json.load(f)
    ans=[]
    for hosptial in lst: 
        ans.append(data[hosptial])
    return json.dumps({k: data[k] for k in lst}, ensure_ascii=False)

In [8]:
@app.route('/load_hospital',methods=['GET'])
def load_hospital():
    import json
    with open('line_hospital.json','r', encoding='UTF-8')as f:
        data=json.load(f)
    
    return json.dumps(data, ensure_ascii=False)

In [9]:
# # 加载模型
# model = torch.nn.Sequential(torch.nn.Linear(784, 128), torch.nn.ReLU(), torch.nn.Linear(128, 10))
# model.load_state_dict(torch.load('mnist_model.pth'))
# model.eval()

@app.route('/upload', methods=['POST'])
def upload_file():
    # 处理文件上传请求
    # if request.method == 'POST':
        # 判断请求中是否包含文件
        if 'file' not in request.files:
            return {'message': 'No file part'}
        file = request.files['file']

        # 如果用户没有选择文件，则返回错误信息
        if file.filename == '':
            return {'message': 'No selected file'}

        # 如果用户上传的文件符合要求，则保存文件并返回成功信息
        if file and isAllowedFile(file.filename):
            # 使用 secure_filename 方法生成安全文件名
            filename = secure_filename(file.filename)
            # 将文件保存到指定目录
            originPath=os.path.join(app.config['UPLOAD_FOLDER'], filename)
            processedPath=os.path.join(app.config['UPLOAD_FOLDER'], "processed"+filename.split('.')[0]+".csv")
            file.save(originPath)
            print("start process data...")
            # 数据预处理并预测
            prcs=Processing()
            prcs.processing(originPath,processedPath)
            print("process data overed")
            # ans=predict()
            # return {'message': ans}
            
            return jsonify({'message': 'File uploaded successfully'})

In [10]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Jun/2023 23:20:08] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [07/Jun/2023 23:20:09] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [07/Jun/2023 23:20:09] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [07/Jun/2023 23:20:20] "GET /hospital HTTP/1.1" 404 -
127.0.0.1 - - [07/Jun/2023 23:20:39] "GET /load_hospital HTTP/1.1" 200 -
